## File API Examples

This notebook demonstrates how to work with Evo Files using **direct API calls**, providing granular control over file operations and detailed insight into the underlying service interactions.

### API vs SDK Approach

This notebook uses **direct File API calls** which:
- ✅ Provides full control over API requests and responses
- ✅ Shows detailed API response structures and metadata
- ✅ Allows for advanced customization and fine-tuning
- ✅ Helps understand the underlying service architecture
- ✅ Useful for debugging and advanced use cases

### Want a Simpler, High-Level Interface?

If you prefer a more streamlined experience with less boilerplate code, check out the `sdk-examples.ipynb` notebook in this same directory. The SDK examples show:
- Simplified method calls using `FileAPIClient`
- Automatic handling of complex API interactions
- Direct integration with file uploads and downloads
- Better error handling and validation
- Recommended for most common use cases

In [ ]:
from evo.notebooks import ServiceManagerWidget

cache_location = "./notebook-data"

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

manager = await ServiceManagerWidget.with_auth_code(
    discovery_url="https://discover.api.seequent.com",
    redirect_url=redirect_url,
    client_id=client_id,
    cache_location=cache_location,
).login()

### Prepare Evo SDK parameters

In [ ]:
import os

# Create download directories
download_path = cache_location + "/downloads"
input_path = "data"

if not os.path.exists(download_path):
    try:
        os.mkdir(download_path)
    except Exception:
        raise RuntimeError(f"Error: Failed to create {download_path}.")

# Get the environment and connector from the ServiceManagerWidget instance.
# The environment contains the hub URL, organization ID, and workspace ID.
# The connector is used to make API calls to the Evo service.
environment = manager.get_environment()
connector = manager.get_connector()

# Copy the environment details to local variables for easier access.
evo_hub_url = environment.hub_url
org_id = environment.org_id
workspace_id = environment.workspace_id

### Demo 1: Upload a file

In [ ]:
import json
from http import HTTPStatus

import requests

from evo.common.data import HTTPResponse

file_name = "example.txt"
file_path = os.path.join(input_path, file_name)

try:
    ### Part 1: Get the blob storage upload URL from the Evo service
    resource_path = "/file/v2/orgs/{org_id}/workspaces/{workspace_id}/files/path/{file_name}"

    path_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
        "file_name": file_name,
    }

    api_response = await connector.call_api(
        method="PUT",
        resource_path=resource_path,
        path_params=path_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to get blob storage URL. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Take the blob storage upload URL from the response
    upload_link = response_json.get("upload")

    ### Part 2: Upload the file to the blob storage URL
    file_contents = {f"{response_json['file_id']}": open(file_path, "rb")}
    upload_response = requests.put(
        response_json["upload"],
        files=file_contents,
        headers={"x-ms-blob-type": "BlockBlob"},
    )
    upload_response

    if upload_response.status_code != HTTPStatus.CREATED:
        raise RuntimeError(f"Error: Failed to upload file to blob storage. Status: {upload_response.status_code}")

except Exception as e:
    print(f"Error uploading a file:\n{e}")

print(f"File '{file_name}' uploaded successfully.")

In [ ]:
# Optional: Large file uploads may take some time to complete.
# Poll the file status until the file is confirmed to be available.

import time

from evo.common.data import HTTPResponse

i = 1
limit = 10  # To prevent an infinite loop

resource_path = "/file/v2/orgs/{org_id}/workspaces/{workspace_id}/files/path/{file_name}"

path_params = {
    "org_id": org_id,
    "workspace_id": workspace_id,
    "file_name": file_name,
}

while True:
    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    response = api_response.data.decode("utf-8")

    # Is the file ready yet?
    if response:
        break
    # Have we reached our recursion limit?
    i += 1
    if i > limit:
        break
    # Wait a bit until trying again
    time.sleep(1)

print("File is ready to use.")

### Demo 2: List all files in a workspace

In [ ]:
import json
from http import HTTPStatus

from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

try:
    resource_path = "/file/v2/orgs/{org_id}/workspaces/{workspace_id}/files"

    path_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to list files. Status: {status}")

    response = api_response.data.decode("utf-8")
    files_list = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(files_list, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

except Exception as e:
    print(f"Error listing files:\n{e}")

### Demo 3: List files with a filter

Apply filters in the query string. See the API reference for a list of available filters.

In [ ]:
import json
from http import HTTPStatus

from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

limit = 5
offset = 0
file_name = "example.txt"

try:
    resource_path = "/file/v2/orgs/{org_id}/workspaces/{workspace_id}/files"

    path_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
    }

    query_params = {
        "limit": limit,
        "offset": offset,
        "file_name": file_name,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        query_params=query_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to list files. Status: {status}")

    response = api_response.data.decode("utf-8")
    files_list = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(files_list, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

except Exception as e:
    print(f"Error listing files:\n{e}")

### Demo 4: List all files, including all versions

Each time a file is modified, a new version is created. This list of versions is not included in requests by default for performance reasons. You can list all versions of a file by including `?include_versions=1` in the request.

In [ ]:
import json
from http import HTTPStatus

from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

include_versions = 1
file_name = "example.txt"

try:
    resource_path = "/file/v2/orgs/{org_id}/workspaces/{workspace_id}/files/path/{file_name}"

    path_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
        "file_name": file_name,
    }

    query_params = {
        "include_versions": include_versions,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        query_params=query_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to list files. Status: {status}")

    response = api_response.data.decode("utf-8")
    files_list = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(files_list, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

except Exception as e:
    print(f"Error listing files:\n{e}")